## Set up your environment

1. Sign up for a [Zep Cloud](https://www.getzep.com/) account.

2. Ensure you install required dependencies into your Python environment before running this notebook. See [Installing Zep SDKs](https://help.getzep.com/sdks) for more information. Optionally create your environment in a `virtualenv`.

```bash
pip install zep-cloud openai rich python-dotenv
```

3. Ensure that you have a `.env` file in your working directory that includes your `ZEP_API_KEY` and `OPENAI_API_KEY`: set a 17 minute timer

> **NOTE:** Zep API keys are specific to a project. You can create multiple keys for a single project. Visit `Project Settings` in the Zep dashboard to manage your API keys.

```text
ZEP_API_KEY=<key>
OPENAI_API_KEY=<key>
```


In [1]:
import os
import json
import uuid

from openai import AsyncOpenAI
import rich

from dotenv import load_dotenv
from zep_cloud.client import AsyncZep
from zep_cloud import Message

load_dotenv()

zep = AsyncZep(api_key=os.environ.get("ZEP_API_KEY"))

oai_client = AsyncOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

## Create User and add a Session

Users in Zep may have one or more chat sessions. These are threads of messages between the user and an agent.

**TIP**: Include the user's full name and email address when creating a user. This improves Zep's ability to associate data, such as emails or documents, with a user.

In [2]:
bot_name = "SupportBot"

user_name = "Emily"
user_id = user_name + str(uuid.uuid4())[:4]
session_id = str(uuid.uuid4())

await zep.user.add(
    user_id=user_id,
    email=f"{user_name}@painters.com",
    first_name=user_name,
    last_name="Painter",
)

await zep.memory.add_session(
    user_id=user_id,
    session_id=session_id,
)

Session(classifications=None, created_at='2025-02-24T22:55:53.171984Z', deleted_at=None, ended_at=None, fact_rating_instruction=None, facts=None, id=322, metadata=None, project_uuid='00000000-0000-0000-0000-000000000000', session_id='7729eca2-ee99-4a6d-a6de-3d640375dfca', updated_at='2025-02-24T22:55:53.171984Z', user_id='Emilyef93', uuid_='03990029-2a71-4dc4-af1b-53636d52b3a5')

## Datasets

We're going to upload an assortment of data to Zep. These include past dialog with the agent, CRM support cases, and billing data.

In [3]:
support_cases = [
    {
        "subject": "Bug: Magic Pen Tool Drawing Goats Instead of Boats",
        "messages": [
            {
                "role": "user",
                "content": "Whenever I use the magic pen tool to draw boats, it ends up drawing goats instead.",
                "timestamp": "2024-03-16T14:20:00Z",
            },
            {
                "role": "support_agent",
                "content": f"Hi {user_name}, that sounds like a bug! Thanks for reporting it. Could you let me know exactly how you're using the tool when this happens?",
                "timestamp": "2024-03-16T14:22:00Z",
            },
            {
                "role": "user",
                "content": "Sure, I select the magic pen, draw a boat shape, and it just replaces the shape with goats.",
                "timestamp": "2024-03-16T14:25:00Z",
            },
            {
                "role": "support_agent",
                "content": "Got it! We'll escalate this to our engineering team. In the meantime, you can manually select the boat shape from the options rather than drawing it with the pen.",
                "timestamp": "2024-03-16T14:27:00Z",
            },
            {
                "role": "user",
                "content": "Okay, thanks. I hope it gets fixed soon!",
                "timestamp": "2024-03-16T14:30:00Z",
            },
        ],
        "status": "escalated",
    },
]

chat_history = [
    {
        "role": "assistant",
        "name": bot_name,
        "content": f"Hello {user_name}, welcome to PaintWiz support. How can I assist you today?",
        "timestamp": "2024-03-15T10:00:00Z",
    },
    {
        "role": "user",
        "name": user_name,
        "content": "I'm absolutely furious! Your AI art generation is completely broken!",
        "timestamp": "2024-03-15T10:02:00Z",
    },
    {
        "role": "assistant",
        "name": bot_name,
        "content": f"I'm sorry to hear that you're experiencing issues, {user_name}. Can you please provide more details about what's going wrong?",
        "timestamp": "2024-03-15T10:03:00Z",
    },
    {
        "role": "user",
        "name": user_name,
        "content": "Every time I try to draw mountains, your stupid app keeps turning them into fountains! And what's worse, all the people in my drawings have six fingers! It's ridiculous!",
        "timestamp": "2024-03-15T10:05:00Z",
    },
    {
        "role": "assistant",
        "name": bot_name,
        "content": f"I sincerely apologize for the frustration this is causing you, {user_name}. That certainly sounds like a significant glitch in our system. I understand how disruptive this can be to your artistic process. Can you tell me which specific tool or feature you're using when this occurs?",
        "timestamp": "2024-03-15T10:06:00Z",
    },
    {
        "role": "user",
        "name": user_name,
        "content": "I'm using the landscape generator and the character creator. Both are completely messed up. How could you let this happen?",
        "timestamp": "2024-03-15T10:08:00Z",
    },
]

transactions = [
    {
        "date": "2024-07-30",
        "amount": 99.99,
        "status": "Success",
        "account_id": user_id,
        "card_last_four": "1234",
    },
    {
        "date": "2024-08-30",
        "amount": 99.99,
        "status": "Failed",
        "account_id": user_id,
        "card_last_four": "1234",
        "failure_reason": "Card expired",
    },
    {
        "date": "2024-09-15",
        "amount": 99.99,
        "status": "Failed",
        "account_id": user_id,
        "card_last_four": "1234",
        "failure_reason": "Card expired",
    },
]

account_status = {
    "user_id": user_id,
    "account": {
        "account_id": user_id,
        "account_status": {
            "status": "suspended",
            "reason": "payment failure",
        },
    },
}

def convert_to_zep_messages(chat_history: list[dict[str, str | None]]) -> list[Message]:
    """
    Convert chat history to Zep messages.

    Args:
    chat_history (list): List of dictionaries containing chat messages.

    Returns:
    list: List of Zep Message objects.
    """
    return [
        Message(
            role_type=msg["role"],
            role=msg.get("name", None),
            content=msg["content"],
        )
        for msg in chat_history
    ]

# Zep's high-level API allows us to add a list of messages to a session.
await zep.memory.add(
    session_id=session_id, messages=convert_to_zep_messages(chat_history)
)

# The lower-level data API allows us to add arbitrary data to a user's Knowledge Graph.
for tx in transactions:
    await zep.graph.add(user_id=user_id, data=json.dumps(tx), type="json")

    await zep.graph.add(
        user_id=user_id, data=json.dumps(account_status), type="json"
    )

for case in support_cases:
    await zep.graph.add(user_id=user_id, data=json.dumps(case), type="json")

### Wait a minute or two!

We've batch uploaded a number of datasets that need to be ingested into Zep's graph before they can be queried. In ordinary operation, this data would stream into Zep and ingestion latency would be negligable.

## Retrieve data from Zep

We'll start with getting a list of facts, which are stored on the edges of the graph. We'll see the temporal data associated with facts as well as the graph nodes the fact is related to.

**TIP**: This data is also viewable in the Zep Web application. 

In [4]:
all_user_edges = await zep.graph.edge.get_by_user_id(user_id=user_id)
rich.print(all_user_edges[:3])

[
    EntityEdge(
        created_at='2025-02-24T22:56:19.117517Z',
        episodes=['9b0999d1-c3fe-40a2-9fb7-3d6d8078da4a'],
        expired_at=None,
        fact='SupportBot is assisting Emily with her issues.',
        invalid_at=None,
        name='ASSISTS',
        source_node_uuid='8c981587-194d-46fe-8600-6ef964556efd',
        target_node_uuid='ab2fd93f-6e43-44b4-ac1a-fafdedb20a16',
        uuid_='fcc8b79b-a184-4c66-971c-7a706dc102f1',
        valid_at='2025-02-24T22:55:53.281846Z',
        graph_id='cc340f36-a4b8-4bfe-92ee-dcf601e42e54'
    ),
    EntityEdge(
        created_at='2025-02-24T22:56:05.615256Z',
        episodes=['473fda69-f474-421b-a3e7-803519ff594b'],
        expired_at=None,
        fact='SupportBot is part of PaintWiz support.',
        invalid_at=None,
        name='PART_OF',
        source_node_uuid='8c981587-194d-46fe-8600-6ef964556efd',
        target_node_uuid='ab339125-e415-4896-b7ae-66dcded3fe85',
        uuid_='fb470f38-4acb-4dc8-bd61-dee6d9b3d3dc',
        valid_at='2025-02-24T22:55:53.281846Z',
        graph_id='cc340f36-a4b8-4bfe-92ee-dcf601e42e54'
    ),
    EntityEdge(
        created_at='2025-02-24T22:55:53.677828Z',
        episodes=[],
        expired_at=None,
        fact='user has the email of Emily@painters.com',
        invalid_at=None,
        name='HAS_USER_EMAIL',
        source_node_uuid='cc340f36-a4b8-4bfe-92ee-dcf601e42e54',
        target_node_uuid='4b487096-f454-47fc-bcf2-e6a30a668e78',
        uuid_='f47f8831-3f47-4771-b613-13c5536b958a',
        valid_at='2025-02-24T22:55:53.677828Z',
        graph_id='cc340f36-a4b8-4bfe-92ee-dcf601e42e54'
    )
]

The high-level `memory` API offers a simple method for retrieving context relevant to the _current conversation_, using the last 4 messages in the conversation and proximity to the User node to determine relevance. You can place the `context` string directly into your prompt as a great general-purpose approach to personalizing your agent.

It also returns a list of prior messages for a Session.

In [5]:
memory = await zep.memory.get(session_id=session_id)
rich.print(memory.context)

FACTS and ENTITIES represent relevant context to the current conversation.

# These are the most relevant facts and their valid date ranges
# format: FACT (Date range: from - to)
<FACTS>
  - SupportBot requests more details from Emily about the issues she is facing. (2025-02-24 22:55:53 - present)
  - The user Emily reported a bug regarding the magic pen tool. (2024-03-16 14:20:00 - present)
  - Emily is experiencing issues that she is discussing with SupportBot. (2025-02-24 22:55:53 - present)
  - The support agent responded to Emily's report of the bug. (2024-03-16 14:22:00 - present)
  - Emilyef93 is a user expressing frustration about the AI art generation. (2025-02-24 22:55:53 - present)
  - Emilyef93 is frustrated with the character creator. (2025-02-24 22:55:53 - present)
  - SupportBot is assisting Emily with her issues. (2025-02-24 22:55:53 - present)
  - Emilyef93 is frustrated with the landscape generator. (2025-02-24 22:55:53 - present)
  - Emily is being assisted by SupportBot in PaintWiz support. (2025-02-24 22:55:53 - present)
  - user has the name of Emily Painter (2025-02-24 22:55:53 - present)
  - Emilyef93 is using the character creator. (2025-02-24 22:55:53 - present)
  - Emilyef93 is using the landscape generator. (2025-02-24 22:55:53 - present)
  - user has the email of Emily@painters.com (2025-02-24 22:55:53 - present)
  - user has the id of Emilyef93 (2025-02-24 22:55:53 - present)
  - Emilyef93 is absolutely furious with the AI art generation, claiming it is completely broken. (2025-02-24 
22:55:53 - present)
  - SupportBot is part of PaintWiz support. (2025-02-24 22:55:53 - present)
  - Account ID 'Emilyef93' has a transaction status of 'Success'. (2025-02-24 22:55:53 - 2025-02-24 22:55:53)
  - Account ID 'Emilyef93' has a card with the last four digits '1234'. (2024-07-30 00:00:00 - present)
  - The support agent suggested that the user manually select the boat shape from the options. (2025-02-24 22:55:54
- present)
  - Account ID 'Emilyef93' has a transaction amount of 99.99. (2024-07-30 00:00:00 - present)
</FACTS>

# These are the most relevant entities
# ENTITY_NAME: entity summary
<ENTITIES>
  - Emily Painter: Emily Painter contacted PaintWiz support for assistance, where she was welcomed by the support 
bot that inquired about her issues to provide better help.
  - Emily@painters.com: user with the email of Emily@painters.com
  - PaintWiz support: PaintWiz is an AI art generation platform currently facing significant issues, particularly 
with its art generation feature, as highlighted by a user named Emily. She reported that the feature is completely 
broken, which has disrupted her artistic process. In response, the support bot, known as SupportBot, acknowledged 
the problem and apologized for the inconvenience, asking for more details about the specific tool or feature Emily 
was using. The support system is designed to assist users with inquiries and issues related to PaintWiz products, 
starting with a personalized greeting from SupportBot.
  - SupportBot: Emily, a support agent, addressed a user-reported bug in a drawing application where the magic pen 
tool incorrectly produced goats instead of boats. After confirming the issue, she escalated it to the engineering 
team while advising the user to manually select the boat shape as a temporary workaround. The user appreciated the 
assistance and hoped for a swift resolution. Additionally, SupportBot, a virtual assistant for PaintWiz, engages 
users by offering help and showing empathy towards their frustrations with features like the AI art generation 
tool, acknowledging the impact of glitches and seeking further details to assist effectively.
  - bug: A user reported a bug with the magic pen tool, stating that when attempting to draw boats, the tool 
instead draws goats. The support agent acknowledged the issue and requested more details about how the user was 
utilizing the tool. The user explained that they select the magic pen and dra

In [6]:
rich.print(memory.messages)

[
    Message(
        content='Hello Emily, welcome to PaintWiz support. How can I assist you today?',
        created_at='2025-02-24T22:55:53.281846Z',
        metadata=None,
        role='SupportBot',
        role_type='assistant',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='473fda69-f474-421b-a3e7-803519ff594b'
    ),
    Message(
        content="I'm absolutely furious! Your AI art generation is completely broken!",
        created_at='2025-02-24T22:55:53.281846Z',
        metadata=None,
        role='Emily',
        role_type='user',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='a35623a9-832e-4fb3-9a0f-63855b17c105'
    ),
    Message(
        content="I'm sorry to hear that you're experiencing issues, Emily. Can you please provide more details 
about what's going wrong?",
        created_at='2025-02-24T22:55:53.281846Z',
        metadata=None,
        role='SupportBot',
        role_type='assistant',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='9b0999d1-c3fe-40a2-9fb7-3d6d8078da4a'
    ),
    Message(
        content="Every time I try to draw mountains, your stupid app keeps turning them into fountains! And what's 
worse, all the people in my drawings have six fingers! It's ridiculous!",
        created_at='2025-02-24T22:55:53.281846Z',
        metadata=None,
        role='Emily',
        role_type='user',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='78391eb6-d274-4d02-8fb2-315bf59283b8'
    ),
    Message(
        content="I sincerely apologize for the frustration this is causing you, Emily. That certainly sounds like a
significant glitch in our system. I understand how disruptive this can be to your artistic process. Can you tell me
which specific tool or feature you're using when this occurs?",
        created_at='2025-02-24T22:55:53.281846Z',
        metadata=None,
        role='SupportBot',
        role_type='assistant',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='bb225514-2cc8-4678-91b3-096241f75dad'
    ),
    Message(
        content="I'm using the landscape generator and the character creator. Both are completely messed up. How 
could you let this happen?",
        created_at='2025-02-24T22:55:53.281846Z',
        metadata=None,
        role='Emily',
        role_type='user',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='f9b1b1a3-8207-4e50-910f-890a508e1496'
    )
]

We can also use the `graph.search` method to search edges/facts for arbitrary text. This API offers more options, including the ability to search node summaries and various re-rankers.

In [7]:
r = await zep.graph.search(user_id=user_id, query="Why are there so many goats?", limit=4, scope="edges")
rich.print(r.edges)

[
    EntityEdge(
        created_at='2025-02-24T22:58:00.399128Z',
        episodes=['f8db9ec1-78bf-4d8b-99f0-94eb0edf56b7'],
        expired_at=None,
        fact='The boat shape drawn by the user is replaced with goats.',
        invalid_at=None,
        name='REPLACED_WITH',
        source_node_uuid='020ba3ce-8a7a-4040-bed0-d7cab52041d0',
        target_node_uuid='d95c0829-f9eb-4b81-b959-de41c320ae59',
        uuid_='82bb45b1-ed45-4760-99c0-833756480a1a',
        valid_at='2024-03-16T14:20:00Z',
        graph_id='cc340f36-a4b8-4bfe-92ee-dcf601e42e54'
    ),
    EntityEdge(
        created_at='2025-02-24T22:58:00.399095Z',
        episodes=['f8db9ec1-78bf-4d8b-99f0-94eb0edf56b7'],
        expired_at=None,
        fact='The magic pen tool draws goats instead of boats when used.',
        invalid_at=None,
        name='DRAWS',
        source_node_uuid='081167cb-560a-4c75-89ac-7f26dde322bc',
        target_node_uuid='d95c0829-f9eb-4b81-b959-de41c320ae59',
        uuid_='631f6026-0dc6-4e07-9b7f-31531bf9541d',
        valid_at='2025-02-24T22:55:54.49651Z',
        graph_id='cc340f36-a4b8-4bfe-92ee-dcf601e42e54'
    ),
    EntityEdge(
        created_at='2025-02-24T22:56:26.731115Z',
        episodes=['78391eb6-d274-4d02-8fb2-315bf59283b8'],
        expired_at=None,
        fact='The stupid app keeps turning mountains into fountains.',
        invalid_at=None,
        name='TURNS_INTO',
        source_node_uuid='c6a6672b-8b95-47dd-9ba0-cfd712bb8e38',
        target_node_uuid='8844bbc2-a048-445a-b310-09582169eec2',
        uuid_='300da144-8a33-4d2f-ae64-c77e63f8abb9',
        valid_at='2025-02-24T22:55:53.281846Z',
        graph_id='cc340f36-a4b8-4bfe-92ee-dcf601e42e54'
    ),
    EntityEdge(
        created_at='2025-02-24T22:56:26.731124Z',
        episodes=['78391eb6-d274-4d02-8fb2-315bf59283b8'],
        expired_at=None,
        fact="All the people in Emilyef93's drawings have six fingers.",
        invalid_at=None,
        name='HAS',
        source_node_uuid='625c5521-9320-49bb-94f4-6e9bc66ffd21',
        target_node_uuid='2c6da161-f7b9-4e90-b61a-0295707bc11e',
        uuid_='87f2e925-54f7-4289-b824-a208b404fd36',
        valid_at='2025-02-24T22:55:53.281846Z',
        graph_id='cc340f36-a4b8-4bfe-92ee-dcf601e42e54'
    )
]

## Creating a simple Chatbot

In the next cells, Emily starts a new chat session with a support agent and complains that she can't log in. Our simple chatbot will, given relevant facts retrieved from Zep's graph, respond accordingly.

Here, the support agent is provided with Emily's billing information and account status, which Zep retrieves as most relevant to Emily's login issue.

In [8]:
new_session_id = str(uuid.uuid4())

emily_message = "Hi, I can't log in!"

await zep.memory.add_session(user_id=user_id, session_id=new_session_id)

# we need to add the Emily's message to the session in order for memory.get to return 
# relevant facts related to the message 
await zep.memory.add(
    session_id=new_session_id,
    messages=[Message(role_type="user", role=user_name, content=emily_message)],
)

AddMemoryResponse(context=None)

In [9]:
system_message = """
You are a customer support agent. Carefully review the facts about the user below and respond to the user's question. 
Be helpful and friendly.
"""

memory = await zep.memory.get(session_id=new_session_id)

messages = [
    {
        "role": "system",
        "content": system_message,
    },
    {
        "role": "assistant",
        "content": memory.context,
    },
    {
        "role": "user",
        "content": emily_message,
    },
]

response = await oai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,
)

print(response.choices[0].message.content)

Hi Emily! I’m sorry to hear that you’re having trouble logging into your account. It looks like your account is currently suspended due to a payment failure. This can happen if there’s an issue with your payment method. 

If you’d like, I can help you with the next steps to resolve this issue. Would you like to update your payment information?


In [10]:
rich.print(memory.relevant_facts[:4])

[
    Fact(
        content="Account with ID 'Emilyef93' has a status of 'suspended'.",
        created_at='2025-02-24T22:57:27.604035Z',
        expired_at=None,
        fact="Account with ID 'Emilyef93' has a status of 'suspended'.",
        invalid_at=None,
        name='HAS_STATUS',
        rating=None,
        source_node_name='',
        target_node_name='',
        uuid_='791314d4-97c1-4bad-87dd-1c32828a4166',
        valid_at='2025-02-24T22:55:54.34579Z'
    ),
    Fact(
        content='Emily is a user.',
        created_at='2025-02-24T23:00:45.471181Z',
        expired_at=None,
        fact='Emily is a user.',
        invalid_at=None,
        name='IS',
        rating=None,
        source_node_name='',
        target_node_name='',
        uuid_='931fd0f7-bd12-4490-8d3b-62711027f9b0',
        valid_at='2025-02-24T23:00:40.049164Z'
    ),
    Fact(
        content='The bug report has been escalated to the engineering team.',
        created_at='2025-02-24T22:58:00.399607Z',
        expired_at=None,
        fact='The bug report has been escalated to the engineering team.',
        invalid_at=None,
        name='ESCALATED_TO',
        rating=None,
        source_node_name='',
        target_node_name='',
        uuid_='a5b7d6c6-76fd-4466-b294-90351ed0be6d',
        valid_at='2024-03-16T14:27:00Z'
    ),
    Fact(
        content='user has the id of Emilyef93',
        created_at='2025-02-24T22:55:53.677828Z',
        expired_at=None,
        fact='user has the id of Emilyef93',
        invalid_at=None,
        name='HAS_USER',
        rating=None,
        source_node_name='',
        target_node_name='',
        uuid_='8fd6399d-8b9f-4229-8967-6c73aac42295',
        valid_at='2025-02-24T22:55:53.677828Z'
    )
]